# MNIST Tensorboard Tutorial

In [1]:
!sudo unlink /usr/local/cuda
!sudo ln -s /usr/local/cuda-7.5 /usr/local/cuda

In [2]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Establish TensorFlow Graph

In [3]:
image_size    = 28
flat_size     = image_size ** 2
n_categories  = 10
learning_rate = 0.01

graph = tf.Graph()
with graph.as_default():
    
    # Input
    x      = tf.placeholder(tf.float32, shape = [None, flat_size])
    labels = tf.placeholder(tf.float32, [None, n_categories])
    
    # Training Variables
    W = tf.Variable(tf.zeros(shape = [flat_size, n_categories]))
    b = tf.Variable(tf.zeros(shape = [n_categories]))
    
    # Model
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    
    # Optimizer
    cross_entropy = -tf.reduce_sum(labels * tf.log(y))
    optimizer     =  tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    
    # Test accuracty of the model
    correct  = tf.equal(tf.argmax(y, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

In [4]:
n_steps = 1000
batch_size = 100

with tf.Session(graph = graph) as session:
    
    session.run(tf.initialize_all_variables())
    for _ in range(n_steps):
        batch_data, batch_labels = mnist.train.next_batch(batch_size)
        feed = {x: batch_data, labels:batch_labels}
        _ = session.run([accuracy, optimizer], feed_dict = feed)
        
    print(accuracy.eval(feed_dict = {x: mnist.test.images, labels: mnist.test.labels}))

0.9081


## Using CNN

In [5]:
# Define helpful functions

def weight_variable(shape):
    '''
    Create a TF variable initialized to random,
    uniformly distributed values of shape *shape*.
    '''
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1))

def bias_variable(shape):
    '''
    Create a TF variable of small
    constant values fo shape *shape*.
    '''
    return tf.Variable(tf.constant(0.1, shape = shape))

def conv2d(x, W):
    '''
    Perform convolution of stride one and zero padding.
    Output will be same size as input
    '''
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def max_pool_2x2(x):
    '''
    Perform max pooling over 2x2 blocks
    '''
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

Adding TensorBoard summaries for the data visualization

In [6]:
# Establish the graph
image_size     = 28
flat_size      = image_size**2
color_channels = 1
n_categories   = 10
learning_rate  = 1e-4

graph = tf.Graph()
with graph.as_default():
    
    ### Inputs
    
    x         = tf.placeholder(tf.float32, shape = [None, flat_size])
    l         = tf.placeholder(tf.float32, shape = [None, n_categories])
    keep_prob = tf.placeholder('float')
    
    ### Variables
    
    # First Layer
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    
    # Second Layer
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    
    # Fully Connected Layer
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    
    # FCL - Softmax Layer
    W_fc2 = weight_variable([1024, n_categories])
    b_fc2 = bias_variable([n_categories])
    
    ### Model
    
    def cnn():
        '''
        The CNN model for the graph.
        '''
        
        # First reshape input data to an image representation
        image = tf.reshape(x, [-1, image_size, image_size, color_channels])
        
        # First Layer
        conv = conv2d(image, W_conv1) + b_conv1
        relu = tf.nn.relu(conv)
        pool = max_pool_2x2(relu)
        
        # Second Layer
        conv = conv2d(pool, W_conv2) + b_conv2
        relu = tf.nn.relu(conv)
        pool = max_pool_2x2(relu)
        
        # FCL
        # Image is now 7x7. Need FCL of 1024 neurons to process entire image.
        # Reshape tensor to batch of vectors then apply regression
        flat = tf.reshape(pool, [-1, 7 * 7 * 64])
        fc   = tf.matmul(flat, W_fc1) + b_fc1
        relu = tf.nn.relu(fc)
        
        # Dropout layer
        # Reduces overfitting
        # keep_prob is a placeholder to allow disabling during testing
        drop = tf.nn.dropout(relu, keep_prob)
        
        # FCL - Softmax
        return tf.nn.softmax(tf.matmul(drop, W_fc2) + b_fc2)
    
    # Name scope to organize nodes in visualizer
    with tf.name_scope('Model') as scope:
        y = cnn()
        print(y.get_shape())
    
    ### Optimizer
    
    with tf.name_scope('CrossEntropy') as scope:
        cross_entropy = -tf.reduce_sum(l * tf.log(y))
        
    with tf.name_scope('Train') as scope:
        optimizer     = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    
    ### Testing
    
    with tf.name_scope('Test') as scope:
        correct  = tf.equal(tf.argmax(y, 1), tf.argmax(l, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    ### Summary Ops
    
    # Model
    with tf.name_scope('Model') as scope:
    
        # First Layer
        wc1_hist = tf.histogram_summary('conv1 weights', W_conv1)
        bc1_hist = tf.histogram_summary('conv1 biases', b_conv1)

        # Second Layer
        wc2_hist = tf.histogram_summary('conv2 weights', W_conv2)
        bc2_hist = tf.histogram_summary('conv2 biases', b_conv2)

        # First FCL
        wf1_hist = tf.histogram_summary('fcl1 weights', W_fc1)
        bf1_hist = tf.histogram_summary('fcl1 biases', b_fc1)

        # Second FCL
        wf2_hist = tf.histogram_summary('fc2 weights', W_fc2)
        bf1_hist = tf.histogram_summary('fc2 biases', b_fc2)
        
    # Cross Entropy
    with tf.name_scope('CrossEntropy') as scope:
        ce_summ  = tf.scalar_summary('cross entropy', cross_entropy)
        
    # Test
    with tf.name_scope('Train') as scope:
        acc_summ = tf.scalar_summary('accuracy', accuracy)
        
    # Merge all summaries
    merged = tf.merge_all_summaries()

(?, 10)


In [7]:
# Run the session
n_steps    = 20000
batch_size = 50
log_step   = 1000
keep       = 0.5
log_dir    = './log/mnist_logs'

with tf.Session(graph = graph) as session:
    
    # Write merged summaries to log_dir
    writer = tf.train.SummaryWriter(log_dir, session.graph)
    
    session.run(tf.initialize_all_variables())
    for i in range(n_steps):
        
        # Run the batch
        batch_data, batch_labels = mnist.train.next_batch(batch_size)
        feed = {x: batch_data, l: batch_labels, keep_prob: keep}
        _ = session.run([optimizer], feed_dict = feed)
        
        # Logging every log_step
        if i % log_step == 0:
            log_feed = {x: batch_data, l: batch_labels, keep_prob: 1.0}
            summ_str, acc = session.run([merged, accuracy], feed_dict = log_feed)
            writer.add_summary(summ_str, i)
            print('Step {0}: training accuracy = {1:g}%'.format(i, acc))
            
    # Test accuracy
    test_feed = {x: mnist.test.images, l: mnist.test.labels, keep_prob: 1.0}
    print('Test accuracy = {0:g}%'.format(accuracy.eval(feed_dict = test_feed)))

Step 0: training accuracy = 0.1%
Step 1000: training accuracy = 0.92%
Step 2000: training accuracy = 0.96%
Step 3000: training accuracy = 0.96%
Step 4000: training accuracy = 1%
Step 5000: training accuracy = 1%
Step 6000: training accuracy = 1%
Step 7000: training accuracy = 1%
Step 8000: training accuracy = 1%
Step 9000: training accuracy = 0.98%
Step 10000: training accuracy = 0.98%
Step 11000: training accuracy = 1%
Step 12000: training accuracy = 1%
Step 13000: training accuracy = 1%
Step 14000: training accuracy = 1%
Step 15000: training accuracy = 1%
Step 16000: training accuracy = 1%
Step 17000: training accuracy = 1%
Step 18000: training accuracy = 1%
Step 19000: training accuracy = 1%
Test accuracy = 0.9919%


In [8]:
!/opt/anaconda/bin/tensorboard --logdir=./log/mnist_logs --port=8088

I tensorflow/stream_executor/dso_loader.cc:108] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:108] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:108] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:108] successfully opened CUDA library libcuda.so locally
I tensorflow/stream_executor/dso_loader.cc:108] successfully opened CUDA library libcurand.so locally
Starting TensorBoard  on port 8088
(You can navigate to http://0.0.0.0:8088)
10.182.71.230 - - [03/Aug/2016 14:53:23] "GET / HTTP/1.1" 200 -
10.182.71.230 - - [03/Aug/2016 14:53:23] "GET /external/plottable/plottable.css HTTP/1.1" 200 -
10.182.71.230 - - [03/Aug/2016 14:53:23] "GET /lib/css/global.css HTTP/1.1" 200 -
10.182.71.230 - - [03/Aug/2016 14:53:23] "GET /external/plottable/plottable.min.js HTTP/1.1" 200 -
10.182.71.230 - - [03/Aug/2016 14:53:23] "GET /external/lodash/lodash.min.js